## 1. Importing Libraries

In [3]:
import time
from configparser import ConfigParser
from fyers_api import fyersModel
import pandas as pd
import os
import talib              #https://cloudstrata.io/install-ta-lib-on-ubuntu-server/
import datetime as dt
import pdb
import asyncio

#### Talib installation in notebbok

! wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

! tar -xzf ta-lib-0.4.0-src.tar.gz

! cd ta-lib/

import getpass
password = getpass.getpass()
command = "sudo ./configure"



import getpass
password = getpass.getpass()
command = "sudo make"

import getpass
password = getpass.getpass()
command = "sudo make install"

! pip install ta-lib

! export LD_LIBRARY_PATH=/usr/local/lib:$LD_LIBRARY_PATH

## 2. Code

In [4]:
config = ConfigParser()
config.read('config.ini')

with open("access.txt", 'r') as r:
    access_token = r.read()

print("access_token:", access_token)


client_id='L0QBIDUTN6-100'
fyers = fyersModel.FyersModel(token=access_token, log_path=os.getcwd(), client_id = client_id)
validation = fyers.get_profile()
print("validation:", validation)

access_token: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDA4MzA1NzgsImV4cCI6MTcwMDg3MjIxOCwibmJmIjoxNzAwODMwNTc4LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFlKMXlfUVZ1M3o0NGxuRjVaRWYwSDluLVZTQnVtekY1MUhTUWphTFVtTVhPY3ZoUmxYZlQ4aS11RldSUUtXT3hlVngxX2g0WDN4WkY5YVJMUlJQd1hJSjZ2a2xBUnlRM1FYLU5HWDM1RFRRWHhhcz0iLCJkaXNwbGF5X25hbWUiOiJUVVNIQVIgVU1FU0ggU0VUSEkiLCJvbXMiOiJLMSIsImhzbV9rZXkiOiJhOGQ5ZTJjYzc1NWJhMmY1YjFiOThlYTkxNDdlY2MyMmZkMjI2ODVkMmM5NWY4YTA0MTk3ODVmYyIsImZ5X2lkIjoiWFQwOTk2MCIsImFwcFR5cGUiOjEwMCwicG9hX2ZsYWciOiJOIn0.uVd3mbHNW_CEmyGSF18iDkl1FY4EQOjnYcS9eY-jrEo
validation: {'s': 'ok', 'code': 200, 'message': '', 'data': {'fy_id': 'XT09960', 'name': 'TUSHAR UMESH SETHI', 'image': None, 'display_name': None, 'pin_change_date': '31-10-2023 20:01:26', 'email_id': 'tusharsethi385@gmail.com', 'pwd_change_date': None, 'PAN': '---------', 'mobile_number': '9665442404', 'totp': Tru

## V1

In [4]:
import time
from fyers_api import fyersModel
import pandas as pd
import talib

# Initialize variables
banknifty_symbol = "NSE:NIFTYBANK-INDEX"
is_long_trade = False
is_short_trade = False
rsi_period = 14
call_rsi_threshold = 60
put_rsi_threshold = 40

# Function to process data and place trades
def process_data(data):
    global is_long_trade, is_short_trade, call_symbol, put_symbol

    df = pd.DataFrame(data['candles'])
    print("DataFrame:")
    print(df)

    df['rsi'] = talib.RSI(df[4], timeperiod=rsi_period)
    print("RSI values:")
    print(df['rsi'])

    print(str(time.localtime().tm_hour) + ':' + str(time.localtime().tm_min) + ':' + str(time.localtime().tm_sec) +
          " RSI (-2candle) :" + str(df['rsi'].iloc[-2]) + " RSI (-1 candle) : " + str(df['rsi'].iloc[-1]))

    # Check for long entry condition
    if not is_long_trade and df['rsi'].iloc[-2] < call_rsi_threshold and df['rsi'].iloc[-1] > call_rsi_threshold:
        # Place long trade
        print("RSI crossover: Buy Bank Nifty / Buy Call Option")
        round_to_strike = int(round(float(df[4].iloc[-1]), -2))
        call_symbol = "NSE:BANKNIFTY23N22" + str(round_to_strike) + 'CE'
        #call_symbol = "NSE:BANKNIFTY231122" + str(round_to_strike) + 'CE'    #old format
        # call_symbol = 'NSE:BANKNIFTY23N2243500PE'      #required format
        trade = {
            "symbol": call_symbol,
            "qty": 15,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "False",
            "stopLoss": 0,
            "takeProfit": 0
        }
        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = True
        is_short_trade = False

    if is_long_trade:
        print("Long entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_long_trade = False

    # Check for short entry
    if not is_short_trade and df['rsi'].iloc[-2] > put_rsi_threshold and df['rsi'].iloc[-1] < put_rsi_threshold:
        # Place short trade
        print("RSI crossover: Sell Bank Nifty / Buy Put Option")
        round_to_strike = int(round(float(df[4].iloc[-1]), -2))
        put_symbol = "NSE:BANKNIFTY23N22" + str(round_to_strike) + 'PE'
        trade = {
            "symbol": put_symbol,
            "qty": 15,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "False",
            "stopLoss": 0,
            "takeProfit": 0
        }

        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = False
        is_short_trade = True

    if is_short_trade:
        print("Short entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_short_trade = False

# Symbol data for historical candles
symbol_data = {
    "symbol": banknifty_symbol,
    "resolution": "1",
    "date_format": "1",
    "range_from": "2023-11-17",
    "range_to": "2023-11-21",
    "cont_flag": "1"
}

while True:
    # Get historical candles data
    data = fyers.history(data=symbol_data)
    process_data(data)
    time.sleep(30)  # Sleep for 30 seconds before the next iteration
    


DataFrame:
               0         1         2         3         4  5
0     1700192700  43656.25  43769.85  43574.10  43749.80  0
1     1700192760  43742.20  43852.00  43729.50  43841.15  0
2     1700192820  43862.15  43862.15  43819.95  43824.40  0
3     1700192880  43822.95  43835.85  43772.00  43795.05  0
4     1700192940  43790.60  43813.40  43779.30  43810.00  0
...          ...       ...       ...       ...       ... ..
1026  1700554080  43723.55  43723.55  43709.15  43711.25  0
1027  1700554140  43711.00  43712.35  43706.00  43709.25  0
1028  1700554200  43708.80  43716.70  43702.75  43716.70  0
1029  1700554260  43718.20  43719.20  43711.00  43713.70  0
1030  1700554320  43714.40  43720.95  43709.15  43712.50  0

[1031 rows x 6 columns]
RSI values:
0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
1026    63.537387
1027    61.958758
1028    65.406654
1029    62.933063
1030    61.924248
Name: rsi, Length: 1031, dtype: fl

KeyboardInterrupt: 

## V2

In [43]:
import time

In [7]:
import time
from fyers_api import fyersModel
import pandas as pd
import talib

# Initialize variables
banknifty_symbol = "NSE:NIFTYBANK-INDEX"
is_long_trade = False
is_short_trade = False
rsi_period = 14
call_rsi_threshold = 60
put_rsi_threshold = 55

# Function to process data and place trades
def process_data(data):
    global is_long_trade, is_short_trade, call_symbol, put_symbol

    df = pd.DataFrame(data['candles'])
    print("DataFrame:")
    df[0] = pd.to_datetime(df[0],unit='s')
    df[0]=(df[0].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df[0] = df[0].dt.tz_localize(None)
    # df= df.set_index('date')
    # print(df)

    df['rsi'] = talib.RSI(df[4], timeperiod=rsi_period)
    print(df)

    print(str(time.localtime().tm_hour) + ':' + str(time.localtime().tm_min) + ':' + str(time.localtime().tm_sec) +
          " RSI (-2candle) :" + str(df['rsi'].iloc[-2]) + " RSI (-1 candle) : " + str(df['rsi'].iloc[-1]))
    

    # Check for long entry condition
    if not is_long_trade and df['rsi'].iloc[-2] < 60 and df['rsi'].iloc[-1] > 60:
        # Place long trade
        print("RSI crossover: Buy Bank Nifty / Buy Call Option")
        round_to_strike = int(round(float(df[4].iloc[-1]), -2))
        call_symbol = "NSE:BANKNIFTY23N31" + str(round_to_strike) + 'CE'
        #call_symbol = "NSE:BANKNIFTY231122" + str(round_to_strike) + 'CE'    #old format
        # call_symbol = 'NSE:BANKNIFTY23N2243500PE'      #required format
        trade = {
            "symbol": call_symbol,
            "qty": 1,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "True",
            "stopLoss": 0,
            "takeProfit": 0
        }
        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = True
        is_short_trade = False

    if is_long_trade:
        print("Long entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_long_trade = False

    # Check for short entry
    if not is_short_trade and df['rsi'].iloc[-2] > put_rsi_threshold and df['rsi'].iloc[-1] < put_rsi_threshold:
        # Place short trade
        print("RSI crossover: Sell Bank Nifty / Buy Put Option")
        round_to_strike = int(round(float(df[4].iloc[-1]), -2))
        put_symbol = "NSE:BANKNIFTY23N" + str(round_to_strike) + 'PE'
        trade = {
            "symbol": put_symbol,
            "qty": 15,
            "type": 2,
            "side": 1,
            "productType": "INTRADAY",
            "limitPrice": 0,
            "stopPrice": 0,
            "validity": "DAY",
            "disclosedQty": 0,
            "offlineOrder": "False",
            "stopLoss": 0,
            "takeProfit": 0
        }

        order = fyers.place_order(data=trade)

        # Print relevant information for debugging
        print('Order parameters:', trade)
        print('Response:', order)

        # Set trade flags
        is_long_trade = False
        is_short_trade = True

    if is_short_trade:
        print("Short entry has been taken. Check conditions for Stop Loss or Target")
        # Write your logic for stop loss
        # is_short_trade = False

# Symbol data for historical candles
symbol_data = {
    "symbol": banknifty_symbol,
    "resolution": "1",
    "date_format": "1",
    "range_from": "2023-11-23",
    "range_to": "2023-11-23",
    "cont_flag": "1"
}

while True:
    # Get historical candles data
    data = fyers.history(data=symbol_data)
    process_data(data)
    time.sleep(30)  # Sleep for 30 seconds before the next iteration
    


DataFrame:
                      0         1         2         3         4  5        rsi
0   2023-11-23 09:15:00  43452.75  43515.75  43451.35  43510.80  0        NaN
1   2023-11-23 09:16:00  43509.75  43517.00  43486.10  43505.65  0        NaN
2   2023-11-23 09:17:00  43507.20  43519.30  43496.75  43514.30  0        NaN
3   2023-11-23 09:18:00  43513.35  43518.70  43499.70  43504.20  0        NaN
4   2023-11-23 09:19:00  43505.95  43516.25  43504.35  43505.75  0        NaN
..                  ...       ...       ...       ...       ... ..        ...
370 2023-11-23 15:25:00  43590.05  43594.15  43587.10  43589.40  0  55.258053
371 2023-11-23 15:26:00  43589.85  43598.15  43585.90  43595.45  0  58.408168
372 2023-11-23 15:27:00  43597.15  43602.10  43595.15  43600.80  0  61.021642
373 2023-11-23 15:28:00  43598.65  43601.40  43594.65  43598.00  0  58.934423
374 2023-11-23 15:29:00  43597.65  43602.25  43580.85  43590.40  0  53.577595

[375 rows x 7 columns]
18:29:30 RSI (-2candle) :58.9

KeyboardInterrupt: 